# Hybrid Classification

While the standard `predict` method in most machine learning libraries directly outputs the predicted class, we'll create a custom function that will output the predicted class based on a threshold.

We'll use the `predict_proba` method to get the probabilities and then apply the threshold to get the predicted class. However, not all models have the `predict_proba` method. For example, the `SGDClassifier` model doesn't have this method.

The hybrid classification will act as such, output a probability of being a phishing URL, then if the probability is below a certain threshold, we'll classify it as a legitimate URL. If the probability is above the threshold, we'll rely on another model to classify it as phishing or legitimate. This other model can be a pre-trained model or any other model that can classify the URL as phishing or legitimate.

Why do we need this hybrid classification? Beside increasing the accuracy of the model, we want to have the lowest false negative rate possible. A false negative is when a legitimate URL is classified as phishing. This can be very harmful to the user as they might not be able to access a legitimate website. We want to avoid this as much as possible.

**Note:** Our positive class is phishing URLs (1) and the negative class is legitimate URLs (0). The model predicts the probability of being a phishing URL.

## Setup

In [ ]:
import random

import numpy as np


def set_seeds(seed: int):
    """ Set seeds for reproducibility. """
    random.seed(seed)
    np.random.seed(seed)

In [ ]:
random_seed = 42
set_seeds(random_seed)

## Load Dataset

In [ ]:
import pandas as pd

X_test = pd.read_csv("test_data.csv")

In [ ]:
# We are going to further split the test set to reduce the number of requests to the pre-trained model
# But, first, we need to balance the classes in the test set
# Get the number of samples per class (based on the minimum number of samples in the test set)
num_samples_per_class = X_test['is_phishing'].value_counts().min()

num_samples_per_class

26882

In [ ]:
# Sample the same number of samples for each class
X_test_A = X_test[X_test['is_phishing'] == 0].sample(n=num_samples_per_class, random_state=random_seed)
X_test_B = X_test[X_test['is_phishing'] == 1].sample(n=num_samples_per_class, random_state=random_seed)

In [ ]:
# Merge the two samples
X_test = pd.concat([X_test_A, X_test_B])
y_test = X_test["is_phishing"]
X_test = X_test.drop(columns=["url", "is_phishing", "tld"])

In [ ]:
# show count of each class
y_test.value_counts()

,count
is_phishing,
0,26882
1,26882


## Load Model

In [ ]:
from joblib import load

pipeline = load('random_forest_with_pipeline_2024-11-23_06:48:35.joblib')
#pipeline = load('random_forest_with_pipeline_2024-11-24_03:17:44.joblib')

## Pre-trained Model

In [ ]:
import torch

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ealvaradob/bert-finetuned-phishing")
model = AutoModelForSequenceClassification.from_pretrained("ealvaradob/bert-finetuned-phishing")

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/845 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
def predict_using_pre_trained_model(url: str):
    inputs = tokenizer(url, return_tensors="pt").to(device)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    return predicted_class

In [ ]:
def predict(X):
    # Set batch size
    batch_size = 1000

    # Initialize list to store predictions
    all_predictions = []

    # Process in batches
    for start_idx in range(0, len(X), batch_size):
        # Get the batch data
        batch_texts = X["domain"].iloc[start_idx:start_idx + batch_size].tolist()

        # Tokenize the batch
        inputs = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            return_tensors="pt"
        ).to(device)

        # Perform inference
        with torch.no_grad():  # Disable gradient calculations
            outputs = model(**inputs)
            logits = outputs.logits
            predicted_classes = torch.argmax(logits, dim=1).tolist()

        # Append predictions
        all_predictions.extend(predicted_classes)

        # Free memory after processing this batch
        del inputs, outputs
        torch.cuda.empty_cache()

        print(f'Batch {start_idx // batch_size + 1} processed')

    return all_predictions

In [ ]:
def custom_predict(data_frame: pd.DataFrame, legitimate_threshold = None, phishing_threshold = None):
    X = data_frame.drop(columns=["domain"], axis=1)
    probs = pipeline.predict_proba(X)[:, 1]  # Probability of being spam
    predictions = pipeline.predict(X)

    ambiguous_indices = []
    ambiguous_domains = []

    for i in range(len(predictions)):
        if legitimate_threshold is None and phishing_threshold is None:
            continue
        if probs[i] >= phishing_threshold:
            predictions[i] = 1
        elif probs[i] <= legitimate_threshold:
            predictions[i] = 0
        else:
            url = data_frame.iloc[i]["domain"]
            ambiguous_indices.append(i)
            ambiguous_domains.append(url)

    print(f"Ambiguous: {len(ambiguous_indices)}")

    if ambiguous_domains:
        batch_predictions = predict(data_frame.iloc[ambiguous_indices])

        # Update predictions with batch predictions
        for idx, pred in zip(ambiguous_indices, batch_predictions):
            predictions[idx] = int(pred)

    return probs, predictions

## Model Evaluation

In [ ]:
import json
from sklearn.metrics import precision_recall_fscore_support
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [ ]:
# Initialize the metrics dictionary to store the evaluation metrics
metrics = {}

In [ ]:
print("Random Forest")
y_proba, y_pred = custom_predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
overall_metrics = precision_recall_fscore_support(y_test, y_pred, average="weighted")

print(f"Accuracy: {accuracy}")
print(f"Precision: {overall_metrics[0]}")
print(f"Recall: {overall_metrics[1]}")
print(f"F1: {overall_metrics[2]}")

Random Forest
Ambiguous: 0
Accuracy: 0.8871549735882747
Precision: 0.8950277517477168
Recall: 0.8871549735882747
F1: 0.8865899168156784


In [ ]:
print("Hybrid")
y_proba, y_pred = custom_predict(X_test, legitimate_threshold=0.35, phishing_threshold=0.5)
accuracy = accuracy_score(y_test, y_pred)
overall_metrics = precision_recall_fscore_support(y_test, y_pred, average="weighted")

print(f"Accuracy: {accuracy}")
print(f"Precision: {overall_metrics[0]}")
print(f"Recall: {overall_metrics[1]}")
print(f"F1: {overall_metrics[2]}")

Hybrid
Ambiguous: 1252
Batch 1 processed
Batch 2 processed
Accuracy: 0.8907447362547429
Precision: 0.8944002003061189
Recall: 0.8907447362547429
F1: 0.8904909925702502


### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(y_test, y_pred, target_names=["legitimate", "phishing"])